# List of Contents

### 01 Importing libraries and data files Q3

### 02 Wrangling Customers Data Q4

### 03 Quality and Consistency Checks Q5

### 04 Combining Data Q6

### 05 Exporting in pkl format Q8

# 01 Importing libraries and data files Q3

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Assign a path
path = r'/Users/dramblys/Desktop/Achievement 4/Instacart Basket Analysis'

In [3]:
# Import orders_products_merged_very_new.pkl for merging later
ords_prods_merge=pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_very_new.pkl'))

In [4]:
# Import customers.csv 
customers=pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

# 02 Wrangling Customers Data Q4

In [5]:
# Check df
customers.shape

(206209, 10)

In [6]:
# Check data type
customers.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [7]:
# Check data sample, column names
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
# Check stats
customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [9]:
# Rename columns for consistency 
customers.rename(columns = {'First Name':'first_name','Surnam':'last_name','Gender':'gender','STATE':'state','Age':'age','n_dependants':'num_of_dependants'},inplace = True)

In [10]:
# Check renamed columns
customers.columns

Index(['user_id', 'first_name', 'last_name', 'gender', 'state', 'age',
       'date_joined', 'num_of_dependants', 'fam_status', 'income'],
      dtype='object')

# 03 Quality and Consistency Checks Q5

In [11]:
# Check for mixed-type columns
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

first_name


In [12]:
# Check missing values
customers.isnull().sum()

user_id                  0
first_name           11259
last_name                0
gender                   0
state                    0
age                      0
date_joined              0
num_of_dependants        0
fam_status               0
income                   0
dtype: int64

In [13]:
# Create subset of missing
missing_fn = customers[customers['first_name'].isnull()== True]

In [14]:
missing_fn

,user_id,first_name,last_name,gender,state,age,date_joined,num_of_dependants,fam_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


There are 11,259 missing first names. Since all the other info is there, and it is more than 5% of rows - i am keeping as is (but changing datatype from mixed).

In [15]:
# Change datatype from mixed
customers['first_name'] = customers['first_name'].astype('str')

In [16]:
# Check
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

In [17]:
# Check for duplicates
dups = customers[customers.duplicated()]

In [18]:
dups

,user_id,first_name,last_name,gender,state,age,date_joined,num_of_dependants,fam_status,income


No duplicates.

# 04 Combining Data Q6

In [19]:
# Check data types for combining (user_id)
ords_prods_merge.dtypes

Unnamed: 0_x                 int64
product_id                   int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
Unnamed: 0.1                 int64
Unnamed: 0_y                 int64
order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
add_to_cart_order            int64
reordered                    int64
_merge                    category
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
average_price              float64
spending_flag               object
median_order_frequency     float64
order_frequency_flag        object
dtype: object

In [20]:
# Drop Unnamed columns
ords_prods = ords_prods_merge.drop(columns = ['Unnamed: 0_x','Unnamed: 0.1','Unnamed: 0_y'])

In [21]:
ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              int64   
 1   product_name            object  
 2   aisle_id                int64   
 3   department_id           int64   
 4   prices                  float64 
 5   order_id                int64   
 6   user_id                 int64   
 7   order_number            int64   
 8   orders_day_of_week      int64   
 9   order_hour_of_day       int64   
 10  days_since_prior_order  float64 
 11  add_to_cart_order       int64   
 12  reordered               int64   
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  average_price           float64 
 21  spendi

In [22]:
# Merge ords_prods with customers
ords_prods_custmrs = ords_prods.merge(customers, on = 'user_id')   

In [23]:
ords_prods_custmrs.shape

(32404859, 33)

In [24]:
ords_prods_custmrs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 33 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              int64   
 1   product_name            object  
 2   aisle_id                int64   
 3   department_id           int64   
 4   prices                  float64 
 5   order_id                int64   
 6   user_id                 int64   
 7   order_number            int64   
 8   orders_day_of_week      int64   
 9   order_hour_of_day       int64   
 10  days_since_prior_order  float64 
 11  add_to_cart_order       int64   
 12  reordered               int64   
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  average_price           float64 
 21  spendi

# 05 Exporting in pkl format Q8

In [25]:
# Export in pickle format Q8
ords_prods_custmrs.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_consumers.pkl'))